In [1]:
import mlflow
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from model import UCCDRNModel
from dataset import MnistDataset
from hydra import initialize, compose
from utils import get_or_create_experiment
from omegaconf import OmegaConf
torch.autograd.set_detect_anomaly(True)

x = np.arange(-1.0, 1.0, 0.1)

def KL_div(y, y_hat):
    y = torch.clamp(y, 1e-15, 1.0)
    y_hat = torch.clamp(y_hat, 1e-15, 1.0)
    return torch.sum(y * torch.log(y/y_hat), dim=1)


def Ldjs(P, Q):
    M = (P+Q)/2.0
    l = 0.5*KL_div(P, M)+0.5*KL_div(Q, M)
    return torch.mean(l)/torch.log(torch.tensor(2.0))


# set random seed
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True


def init_model_and_optimizer(args, model_cfg, device):
    model = UCCDRNModel(model_cfg).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    return model, optimizer


def init_dataloader(args):
    train_dataset_len = args.train_num_steps * args.batch_size
    train_dataset = MnistDataset(
        mode="train",
        num_instances=args.num_instances,
        num_samples_per_class=args.num_samples_per_class,
        digit_arr=list(range(args.ucc_end-args.ucc_start+1)),
        ucc_start=args.ucc_start,
        ucc_end=args.ucc_end,
        length=train_dataset_len,
    )
    val_dataset_len = args.val_num_steps * args.batch_size
    val_dataset = MnistDataset(
        mode="val",
        num_instances=args.num_instances,
        num_samples_per_class=args.num_samples_per_class,
        digit_arr=list(range(args.ucc_end-args.ucc_start+1)),
        ucc_start=args.ucc_start,
        ucc_end=args.ucc_end,
        length=val_dataset_len,
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        shuffle=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        shuffle=False,
    )
    return train_loader, val_loader


def evaluate(model, val_loader, lr_scheduler, device):
    model.eval()
    val_jsd_loss_list = []
    val_ae_loss_list = []
    val_ucc_loss_list = []
    val_acc_list = []
    with torch.no_grad():
        for batch_samples, batch_labels in val_loader:
            batch_samples = batch_samples.to(device)
            batch_labels = batch_labels.to(device)
            batch_labels_one_hot = F.one_hot(batch_labels, num_classes=model.num_classes)

            ucc_logits, reconstruction = model(
                batch_samples, return_reconstruction=True)

            # jsd_loss = Ldjs(ucc_logits, batch_labels_one_hot)
            # val_jsd_loss_list.append(jsd_loss.item())

            ucc_loss = F.cross_entropy(ucc_logits, batch_labels)
            val_ucc_loss_list.append(ucc_loss.item())

            # ae_loss = F.mse_loss(batch_samples, reconstruction)
            # val_ae_loss_list.append(ae_loss.item())

            # acculate accuracy
            # _, batch_labels = torch.max(batch_labels, dim=1)

            _, ucc_predicts = torch.max(ucc_logits, dim=1)
            acc = torch.sum(
                ucc_predicts == batch_labels).item() / len(batch_labels)
            val_acc_list.append(acc)
    return {
        # "eval_ucc_djs_loss": np.round(np.mean(val_jsd_loss_list), 5),
        # "eval_ae_loss": np.round(np.mean(val_ae_loss_list), 5),
        "eval_ucc_loss": np.round(np.mean(val_ucc_loss_list), 5),
        "eval_ucc_acc": np.round(np.mean(val_acc_list), 5)
    }


def train(args, model, optimizer, lr_scheduler, train_loader, val_loader, device):
    model.train()
    step = 0
    best_eval_acc = 0
    best_eval_loss = 2
    for batch_samples, batch_labels in train_loader:
        batch_samples = batch_samples.to(device)
        batch_labels = batch_labels.to(device)
        batch_labels_one_hot = F.one_hot(batch_labels, 4)
        optimizer.zero_grad()
        ucc_logits = model(batch_samples, return_reconstruction=False)
        ucc_loss = F.cross_entropy(ucc_logits, batch_labels)
        # ae_loss = F.mse_loss(batch_samples, reconstruction)
        # djs_loss = Ldjs(ucc_logits, batch_labels_one_hot)
        # ucc_loss, ae_loss, loss = model.compute_loss(batch_samples, batch_labels, ucc_logits, reconstruction, return_losses=True)
        # loss = (1-model.alpha)*ae_loss+model.alpha*ucc_loss
        
        ucc_loss.backward()
        optimizer.step()
        step += 1

        if step % 10 == 0:
            with torch.no_grad():
                metric_dict = {}
                grad_log = {name: torch.mean(param.grad).cpu().item(
                ) for name, param in model.named_parameters() if isinstance(param.grad, torch.Tensor)}
                encoder_grad_log = [grad for name, grad in grad_log.items() if "encoder" in name and "weight" in name]
                if max(encoder_grad_log)<1e-9:
                    break
                mlflow.log_metrics(grad_log, step=step)
                # metric_dict["train_ae_loss"] = np.round(ae_loss.detach().item(), 5)
                _, pred = torch.max(ucc_logits, dim=1)
                accuracy = torch.sum(pred.flatten() == batch_labels.flatten())/len(batch_labels)
                metric_dict["train_ucc_loss"] = np.round(ucc_loss.detach().item(), 5)
                metric_dict["train_ucc_acc"] = np.round(float(accuracy), 5)
                # metric_dict["loss"] = np.round(float(loss), 5)
            mlflow.log_metrics(metric_dict, step=step)
            print(f"Step {step}:", metric_dict)
            # switch to train mode
        if step % args.save_interval == 0:
            eval_metric_dict = evaluate(
                model,
                val_loader,
                lr_scheduler,
                device)
            print(f"Step {step}:" + ",".join([f"{key}: {value}"for key, value in eval_metric_dict.items()]))
            mlflow.log_metrics(eval_metric_dict, step=step)
            # early stop
            eval_acc = eval_metric_dict["eval_ucc_acc"]
            eval_loss = eval_metric_dict["eval_ae_loss"]
            if eval_acc > best_eval_acc or eval_loss <best_eval_loss:
                if eval_loss <best_eval_loss:
                    best_eval_loss = eval_loss
                    
                if eval_acc > best_eval_acc:
                    best_eval_acc = eval_acc
                mlflow.log_metric("best_eval_acc", best_eval_acc, step=step)
                mlflow.log_metric("best_eval_loss", best_eval_loss, step=step)
                mlflow.pytorch.log_model(
                    model,
                    "best_model"
                )
                torch.save(optimizer, "best_optimizer.pth")
                mlflow.log_artifact(
                    "best_optimizer.pth",
                    "best_optimizer.pth"
                )
            if step == 5000:
                break
        model.train()
        
    print("Training finished!!!")


mlflow.set_tracking_uri("mlruns")
run_name = "ucc-check-init-no-decoder"
experiment_id = get_or_create_experiment(experiment_name=run_name)
mlflow.set_experiment(experiment_id=experiment_id)
cfg_name = "train_drn"
with initialize(version_base=None, config_path="../configs"):
    cfg = compose(config_name=cfg_name)
for lower_bound in x:
    for upper_bound in x:
        if lower_bound >= upper_bound:
            continue
        with mlflow.start_run(nested=True):
            mlflow.log_dict(dict(OmegaConf.to_object(cfg)), "config.yaml")
            args = cfg.args
            cfg.model.drn.init_method = "uniform"
            cfg.model.drn.init_lower_bound = float(lower_bound)
            cfg.model.drn.init_upper_bound = float(upper_bound)
            cfg.model.loss.alpha = 1
            print(cfg.model.drn)
            device = torch.device("cuda" if torch.cuda.is_available() else "mps")
            model, optimizer = init_model_and_optimizer(args, cfg, device)
            train_loader, val_loader = init_dataloader(args)
            mlflow.pytorch.log_model(model, "init_model")
            best_acc = train(args, model, optimizer, None,
                            train_loader, val_loader, device)


Training finished!!!
{'num_bins': 11, 'hidden_q': 100, 'num_layers': 2, 'num_nodes': 9, 'init_method': 'uniform', 'init_upper_bound': 0.49999999999999967, 'init_lower_bound': 0.09999999999999976, 'output_bins': 4}
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples


2025/08/22 10:43:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Step 10: {'train_ucc_loss': np.float64(1.39029), 'train_ucc_acc': np.float64(0.1)}
Step 20: {'train_ucc_loss': np.float64(1.38217), 'train_ucc_acc': np.float64(0.4)}
Step 30: {'train_ucc_loss': np.float64(1.38511), 'train_ucc_acc': np.float64(0.3)}
Step 40: {'train_ucc_loss': np.float64(1.39059), 'train_ucc_acc': np.float64(0.15)}
Training finished!!!
{'num_bins': 11, 'hidden_q': 100, 'num_layers': 2, 'num_nodes': 9, 'init_method': 'uniform', 'init_upper_bound': 0.5999999999999996, 'init_lower_bound': 0.09999999999999976, 'output_bins': 4}
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples


2025/08/22 10:44:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Step 10: {'train_ucc_loss': np.float64(1.38549), 'train_ucc_acc': np.float64(0.25)}
Step 20: {'train_ucc_loss': np.float64(1.39736), 'train_ucc_acc': np.float64(0.15)}
Step 30: {'train_ucc_loss': np.float64(1.38285), 'train_ucc_acc': np.float64(0.25)}
Step 40: {'train_ucc_loss': np.float64(1.3919), 'train_ucc_acc': np.float64(0.2)}
Training finished!!!
{'num_bins': 11, 'hidden_q': 100, 'num_layers': 2, 'num_nodes': 9, 'init_method': 'uniform', 'init_upper_bound': 0.6999999999999996, 'init_lower_bound': 0.09999999999999976, 'output_bins': 4}
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples


...

Training finished!!!
{'num_bins': 11, 'hidden_q': 100, 'num_layers': 2, 'num_nodes': 9, 'init_method': 'uniform', 'init_upper_bound': 0.8999999999999996, 'init_lower_bound': 0.6999999999999996, 'output_bins': 4}
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples


2025/08/22 10:53:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Training finished!!!
{'num_bins': 11, 'hidden_q': 100, 'num_layers': 2, 'num_nodes': 9, 'init_method': 'uniform', 'init_upper_bound': 0.8999999999999996, 'init_lower_bound': 0.7999999999999996, 'output_bins': 4}
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples
x_train shape: torch.Size([50000, 1, 28, 28])
50000 train samples
10000 val samples
tensor(0.3105)
tensor(0.1325)
10000 test samples


2025/08/22 10:53:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Step 10: {'train_ucc_loss': np.float64(1.38491), 'train_ucc_acc': np.float64(0.3)}
Step 20: {'train_ucc_loss': np.float64(1.38945), 'train_ucc_acc': np.float64(0.2)}
Training finished!!!
